# Data prep

Based on [a Notebook from kaggle](https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python)

## Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import pickle

pd.set_option('display.max_columns', 100)
%matplotlib inline

## Import datasets

In [ ]:
df_train = pd.read_csv('./sources/train.csv')
df_test = pd.read_csv('./sources/test.csv')

## Data viz

### Check columns

In [ ]:
# All columns
print(df_train.columns)

# Numerical values
num_col = df_train.select_dtypes(exclude=['object']).columns
print(num_col)

# Categorical values
cat_col = df_train.select_dtypes(include=['object']).columns
print(cat_col)

all_col = {
    'num_col': num_col,
    'cat_col': cat_col
}

In [ ]:
# Using Pickle to save the numerical and categorical columns
with open('./columns.pkl', 'wb') as col:
    pickle.dump(all_col, col)

### SalePrice

In [ ]:
# descriptive statistics summary

df_train['SalePrice'].describe()

In [ ]:
# histogram

sns.distplot(df_train['SalePrice'])

#### Relations between SalePrice and most interesting Numerical features (TotalBsmtSF & GrLivArea)

In [ ]:
# Against GrLivArea

sns.scatterplot(x=df_train['GrLivArea'], y=df_train['SalePrice'])

We can see at least two outlayers with a high GrLivArea for a low SalePrice

In [ ]:
# Against TotalBsmtSF

sns.scatterplot(x=df_train['TotalBsmtSF'], y=df_train['SalePrice'])

We can see at least one outlaer with a great TotalBsmtSF and a low SalePrice. <br />
Also that a 0 TotalBsmtSF can have SalePrice

#### Relation between SalePrice and most interesting Categorical features

In [ ]:
# Against OverallQual

sns.boxplot(x='OverallQual', y='SalePrice', data=df_train)

In [ ]:
# Against YearBuilt

sns.boxplot(x='YearBuilt', y='SalePrice', data=df_train)

#### The others...

In [ ]:
# Correlation Matrix

cormat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(cormat, vmax=.8, square=True)

In [ ]:
# saleprice correlation matrix

k = 10  # number of variables for heatmap
cols = cormat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={
                 'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

GarageCar & GarageArea and TotalBsmtSF & 1stFlrSF are strongly correlated two by two so we may want to keep just one feature of each couple

In [ ]:
# scatterplot

sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea',
        'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols], height=2.5)
plt.show()

### Missing Datas

In [ ]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()
           ).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
# dealing with missing data

# Taking off the too many missing datas columns
df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index, 1)

# Taking off the one row missing data in Electrical
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)

# just checking that there's no missing data missing...
df_train.isnull().sum().max()

Do the same job and the test dataset

In [ ]:
# Taking off the same columns as we did for the train set
df_test = df_test.drop((missing_data[missing_data['Total'] > 1]).index, 1)

### Outliers

In [ ]:
# standardizing data

saleprice_scaled = StandardScaler().fit_transform(
    df_train['SalePrice'][:, np.newaxis])
low_range = saleprice_scaled[saleprice_scaled[:, 0].argsort()][:10]
high_range = saleprice_scaled[saleprice_scaled[:, 0].argsort()][-10:]
print('outer range (low) of the distribution:')
print(low_range)
print('\nouter range (high) of the distribution:')
print(high_range)

We check again the scatterplots to look for outliers

In [ ]:
sns.scatterplot(x=df_train['GrLivArea'], y=df_train['SalePrice'])

The two points with high GrLivArea seems to be outliers however, the two highest points of SalePrice seems to be "in the line"

In [ ]:
# deleting points
df_train.sort_values(by='GrLivArea', ascending=False)[:2]
df_train = df_train.drop(df_train[df_train['Id'] == 1299].index)
df_train = df_train.drop(df_train[df_train['Id'] == 524].index)

### Normality exploration

#### SalePrice

In [ ]:
# histogram and normal probability plot
sns.distplot(df_train['SalePrice'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

We see that `'SalePrice'` as a "big tail". We can avoid this by applying a `log` to it

In [ ]:
# applying log transformation
df_train['SalePrice'] = np.log(df_train['SalePrice'])

In [ ]:
# transformed histogram and normal probability plot
sns.distplot(df_train['SalePrice'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

That fits much better to a normal distribution

#### GrLivArea

In [ ]:
# histogram and normal probability plot
sns.distplot(df_train['GrLivArea'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

Same desease, same medecine:

In [ ]:
# data transformation
df_train['GrLivArea'] = np.log(df_train['GrLivArea'])

On the test set:

In [ ]:
df_test['GrLivArea'] = np.log(df_test['GrLivArea'])

In [ ]:
# transformed histogram and normal probability plot
sns.distplot(df_train['GrLivArea'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

#### TotalBsmtSF

In [ ]:
# histogram and normal probability plot
sns.distplot(df_train['TotalBsmtSF'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['TotalBsmtSF'], plot=plt)

Because of the 0 presents in the dataset, we cannot use `log`. <br />
So we create a new binary feature that says if there's a basement or not

In [ ]:
# create column for new variable (one is enough because it's a binary categorical feature)
# if area>0 it gets 1, for area==0 it gets 0
df_train['HasBsmt'] = pd.Series(
    len(df_train['TotalBsmtSF']), index=df_train.index)
df_train['HasBsmt'] = 0
df_train.loc[df_train['TotalBsmtSF'] > 0, 'HasBsmt'] = 1

In [ ]:
# transform data
df_train.loc[df_train['HasBsmt'] == 1,
             'TotalBsmtSF'] = np.log(df_train['TotalBsmtSF'])

In [ ]:
# histogram and normal probability plot
sns.distplot(df_train[df_train['TotalBsmtSF'] > 0]['TotalBsmtSF'], fit=norm)
fig = plt.figure()
res = stats.probplot(
    df_train[df_train['TotalBsmtSF'] > 0]['TotalBsmtSF'], plot=plt)

Same on test set:

In [ ]:
# create column for new variable (one is enough because it's a binary categorical feature)
# if area>0 it gets 1, for area==0 it gets 0
df_test['HasBsmt'] = pd.Series(
    len(df_test['TotalBsmtSF']), index=df_test.index)
df_test['HasBsmt'] = 0
df_test.loc[df_test['TotalBsmtSF'] > 0, 'HasBsmt'] = 1

# transform data
df_test.loc[df_test['HasBsmt'] == 1,
             'TotalBsmtSF'] = np.log(df_test['TotalBsmtSF'])

### homoscedasticity

#### 'SalePrice' and 'GrLivArea'

In [ ]:
# scatter plot
plt.scatter(df_train['GrLivArea'], df_train['SalePrice'])

#### 'SalePrice' with 'TotalBsmtSF'

In [ ]:
# scatter plot
plt.scatter(df_train[df_train['TotalBsmtSF'] > 0]['TotalBsmtSF'],
            df_train[df_train['TotalBsmtSF'] > 0]['SalePrice'])

### Dummy datas for categorical features

To be shure to have the same number of columns on both train and test sets, we concatenate them before using the `get_dummies` function

In [ ]:
df_train['train'] = 1
df_test['train'] = 0

In [ ]:
combined = pd.concat([df_train, df_test], axis=0, sort=False)

combined

We now create the dummy features

In [ ]:
# convert categorical variable into dummy
combined = pd.get_dummies(combined)

We split again in to separates datasets

In [ ]:
df_train = combined[combined['train'] == 1]
df_test = combined[combined['train'] == 0]

In [ ]:
df_train.drop(['train'], axis=1, inplace=True)
df_test.drop(['train'], axis=1, inplace=True)

print(df_train.shape)
print(df_test.shape)

Fill Na in the train set:

In [ ]:
df_train.fillna(0)

And drop the 'SalePrice' column that got into the test set while combining the two sets

In [ ]:
df_test.drop(['SalePrice'], axis=1, inplace=True)

## Export cleaned files

In [ ]:
df_train.to_csv('./sources/clean_train.csv', index=False)
df_test.to_csv('./sources/clean_test.csv', index=False)